Finding a path through a maze is analogous to many common search problems in computer science. Why not literally find a path through a maze to illustrate the bfs, dfs and A* algorithms?

In [27]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt
#from generic_search import dfs, bfs, node_to_path, astar, Node

The maze will be a two-dimensional grid of `Cells`. A `Cell` is an enum with `str` values where `" "` will represent an empty space and `"X"` will represent a blocked space.

In [28]:
class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"

We will need a way to an individual location in the maze. This will simply be a `NamedTuple` with properties representing the row and column of the location in question.

In [29]:
class MazeLocation(NamedTuple):
    row: int
    column: int

In [30]:
foo = MazeLocation(1,2)

### Generating a random maze

The `Maze` class will internally keep track of a grid (a list of lists) representing its state. It will also have instance variables for the number of rows, number of columns, start location, and goal location. Its grid will be randomly filled with blocked cells. 

The generated maze should be fairly sparse so that there is almost always a path from a given starting location to a given goal location. We'll let the caller of a new maze decide on the exact sparseness, but we will provide a default value of 20% blocked. 

When a random number beats the threshold of the `sparseness` parameter in question, we will simply replace an empty space with a wall. If we do this for every possible place in the maze, statistically, the sparseness of the maze as a whole will approximate the `sparseness` parameter supplied.

In [32]:
class Maze:
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2,
                 start: MazeLocation = MazeLocation(0, 0), goal: MazeLocation = MazeLocation(9,9)) -> None:
        # Initialize basic instance variables
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        
        # fill the grid with empty cells
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        # populate the grid with blocked cells
        self._randomly_fill(rows, columns, sparseness)
        # fill the start and goal locations in
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
    
    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
                    
    # Return a nicely formatted version of the maze for printing
    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output
    
    # Checking if a particular MazeLocation has reached the goal
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    
    # How can we move within our maze?
    # The next function can find the possible next locations from a given MazeLocation
    # Since each maze is different, this function is defined within this class.
    def successors (self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations

In [33]:
# Testing...
maze: Maze = Maze()
    
print(maze)

S   X     
          
  XXX     
 X        
    XX    
 X        
X     X  X
      X   
X X       
  X      G



## Depth-first search